In [1]:
import torch
import torchvision 
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
torch.manual_seed(42)
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

train_dataset, val_dataset = random_split(train_dataset, [45000, 5000])

batch_size = 1000

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('NO GPU AVAILABLE ERROR')
    device = torch.device("cpu")
    
train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
alexnet = torchvision.models.alexnet(pretrained = False)
pretrained_alexnet = torchvision.models.alexnet(pretrained = True)
featureextract_alexnet = torchvision.models.alexnet(pretrained = True)

model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

finetune_model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    pretrained_alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

featureextract_model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    featureextract_alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

# freeze feature extraction (13 layers)
for i, param in enumerate(featureextract_model.parameters()):
    if i < 13:
        param.requires_grad = False

In [4]:
def train(epochs, optimizer, loss_fn, train_loader, val_loader, model, name):
    for epoch in range(epochs):
        model.train()
        for i, (data, labels) in enumerate(train_loader):
            data = data.to(device)
            labels = labels.to(device)

            pred = model(data)
            loss = loss_fn(pred, labels)
            loss.backward()
            optimizer.step()
            writer.add_scalar(f"{name} Loss/train", loss, epoch)
            optimizer.zero_grad()
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

        print('\n*************************************')
        print('Validation the model after epoch:', epoch)
        accuracy(model, val_loader, epoch, f"{name} Accuracy/validation")

def accuracy(model, data_loader, epoch, name):
    correct, total = 0, 0
    with torch.no_grad():
        model.eval()
        for _, (data, labels) in enumerate(data_loader):
            data = data.to(device)
            labels = labels.to(device)
            
            pred = model(data)
            for i in range(len(labels)):
                pr = torch.argmax(pred[i], dim=-1)
                if pr == labels[i]:
                    correct += 1
                total += 1
        print(correct, total, correct/total)
        writer.add_scalar(name, correct/total, epoch)

In [5]:
model.to(device)
finetune_model.to(device)
featureextract_model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
epochs = 20

In [6]:
print('Training AlexNet from scratch')
optimizer = torch.optim.Adam(model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, model, "Alexnet")
print('Test Accuracy')
accuracy(model, test_loader, 0, "Alexnet Accuracy/test")

Training AlexNet from scratch
Epoch 1 [45/45] - Loss: 2.2874557971954346
*************************************
Validation the model after epoch: 0
501 5000 0.1002
Epoch 2 [45/45] - Loss: 1.9699366092681885
*************************************
Validation the model after epoch: 1
850 5000 0.17
Epoch 3 [45/45] - Loss: 1.8583990335464478
*************************************
Validation the model after epoch: 2
1237 5000 0.2474
Epoch 4 [45/45] - Loss: 1.7117093801498413
*************************************
Validation the model after epoch: 3
1598 5000 0.3196
Epoch 5 [45/45] - Loss: 1.5909616947174072
*************************************
Validation the model after epoch: 4
1828 5000 0.3656
Epoch 6 [45/45] - Loss: 1.4799968004226685
*************************************
Validation the model after epoch: 5
2184 5000 0.4368
Epoch 7 [45/45] - Loss: 1.3030411005020142
*************************************
Validation the model after epoch: 6
2456 5000 0.4912
Epoch 8 [45/45] - Loss: 1.2612520456

In [7]:
print('Training AlexNet using Fine-Tuning')
optimizer = torch.optim.Adam(finetune_model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, finetune_model, "Alexnet Fine-Tuning")
print('Test Accuracy')
accuracy(finetune_model, test_loader, 0, "Alexnet Fine-Tuning Accuracy/test")

Training AlexNet using Fine-Tuning
Epoch 1 [45/45] - Loss: 1.8595151901245117
*************************************
Validation the model after epoch: 0
1295 5000 0.259
Epoch 2 [45/45] - Loss: 1.6950253248214722
*************************************
Validation the model after epoch: 1
1510 5000 0.302
Epoch 3 [45/45] - Loss: 1.5712940692901611
*************************************
Validation the model after epoch: 2
2083 5000 0.4166
Epoch 4 [45/45] - Loss: 1.3850033283233643
*************************************
Validation the model after epoch: 3
2618 5000 0.5236
Epoch 5 [45/45] - Loss: 1.0774943828582764
*************************************
Validation the model after epoch: 4
3067 5000 0.6134
Epoch 6 [45/45] - Loss: 1.1105445623397827
*************************************
Validation the model after epoch: 5
3080 5000 0.616
Epoch 7 [45/45] - Loss: 0.9181387424468994
*************************************
Validation the model after epoch: 6
3448 5000 0.6896
Epoch 8 [45/45] - Loss: 0.7831

In [8]:
print('Training AlexNet using Feature Extraction')
optimizer = torch.optim.Adam(featureextract_model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, featureextract_model, "Alexnet Feature Extraction")
print('Test Accuracy')
accuracy(featureextract_model, test_loader, 0, "Alexnet Feature Extraction Accuracy/test")

Training AlexNet using Feature Extraction
Epoch 1 [45/45] - Loss: 1.3259660005569458
*************************************
Validation the model after epoch: 0
2726 5000 0.5452
Epoch 2 [45/45] - Loss: 1.1977894306182861
*************************************
Validation the model after epoch: 1
2885 5000 0.577
Epoch 3 [45/45] - Loss: 1.1058943271636963
*************************************
Validation the model after epoch: 2
2952 5000 0.5904
Epoch 4 [45/45] - Loss: 1.1222182512283325
*************************************
Validation the model after epoch: 3
2950 5000 0.59
Epoch 5 [45/45] - Loss: 1.1288565397262573
*************************************
Validation the model after epoch: 4
2977 5000 0.5954
Epoch 6 [45/45] - Loss: 1.1508796215057373
*************************************
Validation the model after epoch: 5
3016 5000 0.6032
Epoch 7 [45/45] - Loss: 1.0017836093902588
*************************************
Validation the model after epoch: 6
3035 5000 0.607
Epoch 8 [45/45] - Loss: 

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 5, 1, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5, 1, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # 28x28 > 14x14
        x = self.pool(self.relu(self.conv2(x))) # 14x14 > 7x7
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
    
# Create an instance of the model
model_mnist = Model()
model_mnist.to(device)

Model(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
  (relu): ReLU()
)

In [10]:
train_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
train_dataset_mnist, val_dataset_mnist = random_split(train_dataset_mnist, [55000, 5000])
test_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 1000
epochs = 10

train_loader_mnist = DataLoader(train_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=True)
val_loader_mnist = DataLoader(val_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=True)

test_loader_mnist = DataLoader(test_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=False)

In [11]:
print('Training CNN on MNIST dataset')
optimizer = torch.optim.Adam(model_mnist.parameters(), lr=0.001)
train(epochs, optimizer, loss_fn, train_loader_mnist, val_loader_mnist, model_mnist, "CNN for MNIST")
print('Test Accuracy:')
accuracy(model_mnist, test_loader_mnist, 0, "CNN for MNIST Accuracy/test")

Training CNN on MNIST dataset
Epoch 1 [55/55] - Loss: 0.32087275385856635
*************************************
Validation the model after epoch: 0
4570 5000 0.914
Epoch 2 [55/55] - Loss: 0.18869473040103912
*************************************
Validation the model after epoch: 1
4759 5000 0.9518
Epoch 3 [55/55] - Loss: 0.13600088655948646
*************************************
Validation the model after epoch: 2
4840 5000 0.968
Epoch 4 [55/55] - Loss: 0.10418694466352463
*************************************
Validation the model after epoch: 3
4861 5000 0.9722
Epoch 5 [55/55] - Loss: 0.057641424238681795
*************************************
Validation the model after epoch: 4
4884 5000 0.9768
Epoch 6 [55/55] - Loss: 0.061839338392019275
*************************************
Validation the model after epoch: 5
4890 5000 0.978
Epoch 7 [55/55] - Loss: 0.057130035012960434
*************************************
Validation the model after epoch: 6
4898 5000 0.9796
Epoch 8 [55/55] - Loss: 0

In [12]:
!pip install scipy

In [13]:
def runtest(model, test_loader):
    correct, total = 0, 0
    for _, (data, labels) in enumerate(test_loader):
        data = torchvision.transforms.Resize((28,28))(torchvision.transforms.Grayscale()(data))
        data = data.to(device)
        labels = labels.to(device)

        pred = model(data)
        for i in range(len(labels)):
            pr = torch.argmax(pred[i], dim=-1)
            if pr == labels[i]:
                correct += 1
            total += 1
    print(correct, total, correct/total)
    writer.add_scalar("CNN for MNIST with SVHN data Accuracy/test", correct/total, 0)

In [14]:
test_dataset_svhn = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=ToTensor())

test_loader_svhn = DataLoader(test_dataset_svhn,
                        batch_size=batch_size,
                        shuffle=False)

print('Test Accuracy (SVHN):')
runtest(model_mnist, test_loader_svhn)

Using downloaded and verified file: ./data\test_32x32.mat
Test Accuracy (SVHN):
5342 26032 0.20520897357098955
